# Projetos-Problema-de-negocio-Wine-Vinho-21-setembro-2023

## Prever a qualidade de vinho, dado as sua características aferidas

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install -q huggingface_hub

In [3]:
!pip install -q joblib

In [4]:
%%time
from huggingface_hub import hf_hub_url, cached_download
import joblib
import pandas as pd

REPO_ID = "julien-c/wine-quality"
FILENAME = "sklearn_model.joblib"


model = joblib.load(cached_download(
    hf_hub_url(REPO_ID, FILENAME)
))
model

CPU times: user 1.22 s, sys: 194 ms, total: 1.41 s
Wall time: 1.82 s


Pipeline(steps=[('scaler', StandardScaler()), ('SVM', SVC())])

In [5]:
REPO_ID

'julien-c/wine-quality'

In [6]:
data_file = cached_download(
    hf_hub_url(REPO_ID, "winequality-red.csv")
)
!ls -ilha *.csv

 7092361 -rw-rw-r-- 1 valencar valencar 2,0M mai 16 11:13  brasil2023.csv
13640938 -rw-r--r-- 1 valencar valencar 111K mai 10  2022  covid19-22-janeiro-2022.csv
13637747 -rw-r--r-- 1 valencar valencar  90K mai 10  2022  covid19-23-dez-2021.csv
 7118770 -rw-rw-r-- 1 valencar valencar 2,2K mai 18 11:56  estados.csv
 7119940 -rw-rw-r-- 1 valencar valencar 1,9K set 11 21:28  estados_idh2.csv
 7119935 -rw-rw-r-- 1 valencar valencar 1,9K set  5 12:06 'estados_idh (Conflitos entre maiúsculas e minúsculas 1).csv'
 7119934 -rw-rw-r-- 1 valencar valencar 1,9K set  5 12:05 'estados_idh (Conflitos entre maiúsculas e minúsculas).csv'
 7119123 -rw-r--r-- 1 valencar valencar 1,9K set  4 18:07  Estados_idh.csv
13530010 -rw-r--r-- 1 valencar valencar  30K mai 10  2022  mapa_mundo_covid19-4-fevereiro-2022.csv
 7092360 -rw-rw-r-- 1 valencar valencar 5,7M mai 16 11:16  prouni.csv
11954288 -rw-r--r-- 1 valencar valencar  242 mai 10  2022  ratings_saida.csv
 7254889 -rw-r--r-- 1 valencar valencar  90K set 2

In [7]:
data_file

'/home/valencar/.cache/huggingface/hub/f3aec5dda07574be505d0067cc4cadea8b8224353f8a987c809664443c6f6352.1e69d60fe81c6951c1cba05ec464e9f3bfc20f391de2b2e40d0b6573081b69b3'

In [8]:
#head `/home/valencar/.cache/huggingface/hub/f3aec5dda07574be505d0067cc4cadea8b8224353f8a987c809664443c6f6352.1e69d60fe81c6951c1cba05ec464e9f3bfc20f391de2b2e40d0b6573081b69b3`

In [9]:
winedf = pd.read_csv(data_file, sep=";")

print(len(winedf))
winedf.head()

1599


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [10]:
winedf.quality.unique()

array([5, 6, 7, 4, 8, 3])

In [11]:
winedf.quality.value_counts()

quality
5    681
6    638
7    199
4     53
8     18
3     10
Name: count, dtype: int64

In [12]:
winedf.quality.value_counts() / len(winedf) * 100

quality
5    42.589118
6    39.899937
7    12.445278
4     3.314572
8     1.125704
3     0.625391
Name: count, dtype: float64

## As classes estão DESBALANCEADAS

## Transformação de Dados
### classe 4 e classe 3 tem poucos dados -> colocar as tuplas (registros/linha) na classe 5

In [13]:
df = winedf.copy()
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [14]:
# df['quality'] = df['quality'].apply(lambda x : 5 if x == 4 else x) 
# df['quality'].unique()

In [15]:
# df['quality'] = df['quality'].apply(lambda x : 5 if x == 3 else x) 
# df['quality'].unique()

In [16]:
df['quality'].value_counts()

quality
5    681
6    638
7    199
4     53
8     18
3     10
Name: count, dtype: int64

In [17]:
# df['quality'] = df['quality'].apply(lambda x : 7 if x == 8 else x) 
# df['quality'].unique()

In [18]:
df['quality'].value_counts()

quality
5    681
6    638
7    199
4     53
8     18
3     10
Name: count, dtype: int64

In [19]:
df['quality'].value_counts() / len(df) * 100

quality
5    42.589118
6    39.899937
7    12.445278
4     3.314572
8     1.125704
3     0.625391
Name: count, dtype: float64

In [20]:
winedf.quality.value_counts() / len(winedf) * 100

quality
5    42.589118
6    39.899937
7    12.445278
4     3.314572
8     1.125704
3     0.625391
Name: count, dtype: float64

In [21]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [22]:
df[ df.quality == 7 ][:6]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7
16,8.5,0.28,0.56,1.8,0.092,35.0,103.0,0.9969,3.30,0.75,10.5,7
37,8.1,0.38,0.28,2.1,0.066,13.0,30.0,0.9968,3.23,0.73,9.7,7
62,7.5,0.52,0.16,1.9,0.085,12.0,35.0,0.9968,3.38,0.62,9.5,7
128,8.0,0.59,0.16,1.8,0.065,3.0,16.0,0.9962,3.42,0.92,10.5,7


# Separação dos dados de entrada (X) e dado de Saída (y)

In [23]:
y = winedf["quality"]
X = winedf.drop(["quality"], axis=1)

X.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


In [24]:
y.head()

0    5
1    5
2    5
3    6
4    5
Name: quality, dtype: int64

In [25]:
model

Pipeline(steps=[('scaler', StandardScaler()), ('SVM', SVC())])

In [26]:
X[:3]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8


In [27]:
labels = model.predict(X[:3])

In [28]:
labels

array([5, 5, 5])

# Vamos fazer o nosso modelo

In [29]:
X.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


In [30]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=300)
clf

AdaBoostClassifier(n_estimators=300)

In [31]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(
          max_depth=1,
          random_state=42)
tree = tree.fit(X, y)

In [32]:
from sklearn.svm import SVC

my_base_model= SVC()
ada = AdaBoostClassifier(
           base_estimator= my_base_model, #my_base_modeltree,
           n_estimators=500,
           learning_rate=0.5,
           algorithm='SAMME',
           random_state=42)

# Treinar o modelo com dados de entrada (X) e atributo de saída (y)

In [ ]:
%%time

modelo = ada.fit(X,y)
modelo

In [ ]:
labels = modelo.predict(X[:3])
labels

# Verificar a acurácia - taxa de acertos

In [ ]:
model

## comparando os 2 modelos

In [ ]:
%%time

y_prev_model_SVC = model.predict(X)
y_prev_model_Adaboost = modelo.predict(X)

In [ ]:
%%time

from sklearn.metrics import accuracy_score

acc_SVC = accuracy_score(y, y_prev_model_SVC)
acc_Adaboost = accuracy_score(y, y_prev_model_Adaboost)

print('Acurácia do modelo SVC :', acc_SVC)
print('Acurácia do modelo Adaboost :', acc_Adaboost)

In [ ]:
modelo.get_params()

In [ ]:
modelo.get_metadata_routing()

In [ ]:
modelo.base_estimator_

In [ ]:
modelo.estimator_weights_

In [ ]:
modelo.estimator_errors_

In [ ]:
modelo.feature_importances_

In [ ]:
X.head()

In [ ]:
atributos = list(X.columns.values)
print(atributos)

In [ ]:
modelo.feature_importances_

In [ ]:
dfi = pd.DataFrame( {'atributos' : atributos, 'importancia':  modelo.feature_importances_ })
dfi

In [ ]:
dfo = dfi.sort_values(by='importancia', ascending=False).reset_index(drop=True)
dfo

In [ ]:
atributos_importantes = dfo[:5]
atributos_importantes 

In [ ]:
atributos_mais_importantes = list(atributos_importantes.atributos)
atributos_mais_importantes 

In [ ]:
X.tail()

In [ ]:
X[ X.index == 1597]

In [ ]:
x1 = X[ X.index == 1597].values
x1

In [ ]:
import numpy as np

In [ ]:
modelo.predict(x1)

# Deploy da Parte1 
## Salvar o modelo treinado

In [ ]:
import pickle as pkl

filehandler = open("modeloAdaboost.pkl" ,"wb")
pkl.dump(modelo,filehandler)

!ls -lah *.pkl

In [ ]:
!head modeloAdaboost.pkl

# Carregar o modelo treinado

In [ ]:
file = "modeloAdaboost.pkl"
with open(file, 'rb') as f:
    modelo_lido = pkl.load(f)
    
modelo_lido

In [ ]:
modelo_lido.predict(x1)[0]